# Introduction to TensorFlow for Artificial Intelligence, Machine Learning, and Deep Learning

## A primer in machine learning

- traditional programming: rule + data => answers
- machine learning: answers + data => rule

## Hello world example 

- How would you infer the equation like below by training neural network on input data?
```
float hw_function(float x){
    float y = (2 * x) - 1;
    return y;
}
```

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

# define and compile the neural network
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')

# provide the data
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

# train the neural network
model.fit(xs, ys, epochs=500)
print(model.predict([10.0]))